This is a recreation of Example 1 of this paper: https://www.sciencedirect.com/science/article/pii/0098135488800103

In [8]:
from sys import executable
from pyomo.environ import *
from pyomo.core import *
import numpy as np
import matplotlib.pyplot as plt


In [9]:
d1 = 2.22
d2 = 0.733
theta_nom = 2
theta_plus = 2
theta_minus = 2
bigM = 10**6
n = 1 # number of control variables

In [10]:
m = ConcreteModel()

J = range(3) # number of constraints
m.J = Set(initialize = J)

I = range(2) # number of design variables
m.I = Set(initialize = I)

# design variables -- fix in the first steps and then unfix later
m.d = Var(m.I, doc = 'the design variables')
m.d[0].fix(d1)
m.d[1].fix(d2)

# control variables
m.z = Var(doc = 'the control variables')

m.theta = Var(doc = 'the uncertain parameter')

# flex test variables

m.delta = Var(domain = Reals, doc = 'the flexibility index')

m.s = Var(m.J, domain = NonNegativeReals, doc = 'slack for constraint j')

m.y = Var(m.J, domain = Binary, doc = 'whether constraint j is activated')

m.lambd = Var(m.J, domain = NonNegativeReals, doc = 'lagrangian for constraint j')

In [11]:
# Constraints

def f1_rule(m):
    return m.z - m.theta + m.d[0] - 3*m.d[1] + m.s[0] == 0
m.f1 = Constraint(rule = f1_rule)

def f2_rule(m):
    return -m.z - m.theta/3 + m.d[1] + 1/3 + m.s[1] == 0
m.f2 = Constraint(rule = f2_rule)

def f3_rule(m):
    return m.z + m.theta - m.d[0] - 1 + m.s[2] == 0
m.f3 = Constraint(rule = f3_rule)

rule_list = [m.f1, m.f2, m.f3] # collect list of design constraints

m.sum_lambda = Constraint(rule = (sum(m.lambd[j] for j in m.J) == 1))
m.langrangian = Constraint(rule = (m.lambd[0] - m.lambd[1] + m.lambd[2] == 0))

m.y_constraint = ConstraintList()
for j in m.J:
    m.y_constraint.add(m.lambd[j] - m.y[j] <= 0)
    m.y_constraint.add(m.s[j] - bigM*(1-m.y[j]) <= 0)

m.active_constraints = Constraint(rule = (sum(m.y[j] for j in m.J) == n + 1))

def theta_constraint_ub_rule(m):
    return m.theta <= theta_nom + m.delta*theta_plus
m.theta_constraint_ub = Constraint(rule = theta_constraint_ub_rule)

def theta_Constraint_lb_rule(m):
    return m.theta >= theta_nom - m.delta*theta_minus
m.theta_constraint_lb = Constraint(rule = theta_Constraint_lb_rule)

m.integer_cut_Con = ConstraintList(doc = 'integer cut rules')

m.objective = Objective(expr = (m.delta), sense = minimize)

opt = SolverFactory('gurobi')
# opt = SolverFactory('gurobi', solver_io = 'python')
result = opt.solve(m, tee=True)

Set parameter TokenServer to value "coe-vtls1.engr.tamu.edu"
Read LP format model from file C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpc23pok3s.pyomo.lp
Reading time = 0.00 seconds
x1: 14 rows, 12 columns, 34 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 12 columns and 34 nonzeros
Model fingerprint: 0x88278b9e
Variable types: 9 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 1e+06]
Presolve removed 11 rows and 9 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 7 nonzeros
Variable types: 2 continuous, 1 integer (1 binary)
Found heuristic solution: objective 0.5922500

Explored 0 nodes (0 simplex iterations)

In [12]:
# fetch the first set of duals

# redefine the binary as reals to make this LP
m.y.domain = NonNegativeReals

# LP relaxation, deactivate non-relevant constraints
for j in J:
    m.s[j].fix(0)
    m.y[j].fix(m.y[j].value)
    m.lambd[j].fix(m.lambd[j].value)
    if m.y[j].value == 0:
        rule_list[j].deactivate()

m.sum_lambda.deactivate()
m.langrangian.deactivate()
m.active_constraints.deactivate()
m.y_constraint.deactivate()
m.integer_cut_Con.deactivate()

m.dual = Suffix(direction = Suffix.IMPORT)

lp_opt = SolverFactory('gurobi', solver_io = 'python')
result = lp_opt.solve(m, tee=True)

Set parameter QCPDual to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 4 rows, 14 columns and 8 nonzeros
Model fingerprint: 0x89fa1c9d
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 2e+00]
  RHS range        [2e-02, 2e+00]
Presolve removed 4 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.9225000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.922500000e-01


In [13]:
m.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

7 Var Declarations
    d : the design variables
        Size=2, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  2.22 :  None :  True :  True :  Reals
          1 :  None : 0.733 :  None :  True :  True :  Reals
    delta : the flexibility index
        Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :  None : 0.5922499999999998 :  None : False : False :  Reals
    lambd : lagrangian for constraint j
        Size=3, Index=J
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.5 :  None :  True :  True : NonNegativeReals
          1 :     0 :   0.5 :  None :

In [14]:
error

NameError: name 'error' is not defined

In [ ]:
# Set up iteration data collection

i = 1
delta = value(m.objective)
goal = 1

dual_dict = {}
delta_dict = {}
lambd_dict = {}
y_dict = {}

delta_dict[i] = delta
lambd_dict[i] = list(m.lambd[j].value for j in J)
y_dict[i] = list(m.y[j].value for j in J)

keys = list(str(c) for c in m.component_objects(Constraint, active=True))
values = list(m.dual[c[index]] for c in m.component_objects(Constraint, active=True) for index in c )
dual_dict[i] = dict(zip(keys, values))


In [ ]:
dual_dict

In [ ]:
while delta < goal:
    # reverse everything I did before
    m.y.domain = Binary

    for j in J:
        m.s[j].unfix()
        m.y[j].unfix()
        m.lambd[j].unfix()
        if m.y[j].value == 0:
            rule_list[j].activate()

    m.sum_lambda.activate()
    m.langrangian.activate()
    m.active_constraints.activate()
    m.y_constraint.activate()
    m.integer_cut_Con.activate()

    m.dual.deactivate()

    # add the integer cut
    m.integer_cut_Con.add(sum(m.y[j] for j in m.J if m.y[j].value == 1) - sum(m.y[j] for j in m.J if m.y[j].value == 0) <= n)

    # solve for the next active set
    opt = SolverFactory('gurobi', solver_io = 'python')
    result = opt.solve(m, tee=False)
    i = i + 1

    # store results
    lambd_dict[i] = list(m.lambd[j].value for j in J)
    y_dict[i] = list(m.y[j].value for j in J)
    delta = value(m.objective)
    delta_dict[i] = delta

    # find the duals for this active set
    # redefine the binary as reals to make this LP
    m.y.domain = NonNegativeReals

    # LP relaxation, deactivate non-relevant constraints
    for j in J:
        m.s[j].fix(0)
        m.y[j].fix(m.y[j].value)
        m.lambd[j].fix(m.lambd[j].value)
        if m.y[j].value == 0:
            rule_list[j].deactivate()

    m.sum_lambda.deactivate()
    m.langrangian.deactivate()
    m.active_constraints.deactivate()
    m.y_constraint.deactivate()
    m.integer_cut_Con.deactivate()
    m.dual.activate()

    lp_opt = SolverFactory('gurobi', solver_io = 'python')
    result = lp_opt.solve(m, tee=False)

    keys = list(str(c) for c in m.component_objects(Constraint, active=True))
    values = list(m.dual[c[index]] for c in m.component_objects(Constraint, active=True) for index in c)
    dual_dict[i] = dict(zip(keys, values))


# leave behind the duals for the active constraints only
for key in dual_dict:
    del dual_dict[key]['theta_constraint_ub']
    del dual_dict[key]['theta_constraint_lb']

# reactivate the fs, deactivate the unimportant stuff so we only have the fs
for j in J:
    rule_list[j].activate()

m.theta_constraint_ub.deactivate()
m.theta_constraint_lb.deactivate()
m.z.fix(m.z.value)
m.theta.fix(m.theta.value)
m.delta.fix(m.delta.value)

# unfix the design variables
m.d.unfix()

In [ ]:
dual_dict

In [ ]:
# fetch A matrix off of active constraints

m.write('model.lp', io_options={'symbolic_solver_labels': True})

import gurobipy as GBP

gurobi_model = GBP.read('model.lp')
A = gurobi_model.getA()
A = A.toarray()[:,1:]
# f1 is row 0, f2 is row 1, f3 is row 2
# d(0) is column 1, d(1) is column 2
# A only has the active constraints

In [ ]:
# extract the duals in matrix form

list_of_keys = ['f{}'.format(j+1) for j in J]

dual_matrix = np.zeros((len(y_dict), len(J)), dtype='float')

for k in range(len(y_dict)):
    for i in range(n+1):
        dual_key = list(dual_dict[k+1].keys())[i]
        for j in J:
            if dual_key == list_of_keys[j]:
                dual_matrix[k][j] = dual_dict[k+1][dual_key]


In [ ]:
# get sensitivity coeffs

sigma_matrix = np.empty((len(y_dict), len(I)), dtype='float')
for k in range(len(y_dict)):
    for i in I:
        sigma_matrix[k][i] = - sum(dual_matrix[k][j] * A[j][i] for j in J)

In [ ]:
sigma_matrix

In [ ]:
# calculate capacity change needed for minimum investment to reach goal

p3 = ConcreteModel()

p3.I = Set(initialize = I)

beta = [10,10] # price of improvement

p3.goal = Var(domain = NonNegativeReals, doc = 'flex index goal')
p3.delta = Var(p3.I, domain = NonNegativeReals, doc = 'design 1 capacity change')

p3.Constraint = ConstraintList()
for k in range(len(y_dict)):
    p3.Constraint.add(delta_dict[k+1] + sum(sigma_matrix[k][i]*p3.delta[i] for i in p3.I) >= p3.goal)
for i in p3.I:
    p3.Constraint.add(p3.delta[i] >= 0)
p3.Constraint.add(p3.goal >= list(delta_dict.items())[0][1])
p3.Constraint.add(p3.goal <= list(delta_dict.items())[-1][1])

p3.objective = Objective(expr = (sum(beta[i] * p3.delta[i] for i in p3.I)), sense = minimize)
# opt = SolverFactory('gurobi', solver_io = 'python')

p3.write('p3.lp', io_options={'symbolic_solver_labels': True})


Solve this parametrically to get the cost vs flex analysis curve manually

In [ ]:
# get the above model's matrix
import gurobipy as GBP

gurobi_model = GBP.read('p3.lp')
A = gurobi_model.getA()
A = A.toarray()

# # find the stuff that's set to maximize, flip them to minimize

sense = np.array(gurobi_model.getAttr("Sense",gurobi_model.getConstrs()))
A[sense == '>',:] = A[sense == '>',:] * -1

# get the F matrix for theta coeffs in RHS of constraints. Need to flip em because they go on the other side of the equation by default
F = A[:-2,-1] * -1
F = F.reshape(-1,1)

# get CRa, the coeffs for UB + LB of thetas
CRa = A[-2:,-1]
CRa = CRa.reshape(-1,1)

# get just the d coefficients
A = A[:-2,:-1]

In [ ]:
# get the b

b = gurobi_model.getAttr('RHS', gurobi_model.getConstrs())
b = np.array(b)
b[sense == '>'] = b[sense == '>'] * -1

# pull out the bounds for the thetas
CRb = b[-2:]
CRb = CRb.reshape(-1,1)

# pull out the constant RHS for the other constraints
b = b[:-2]
b = b.reshape(-1,1)

In [ ]:
# pull out the objective coefficients in terms of just the design variables

c = gurobi_model.getAttr('Obj', gurobi_model.getVars())
c = c[:-1]
c = np.array(c)
c = c.reshape(-1,1)

In [ ]:
H = np.zeros((A.shape[1],F.shape[1])) # if we have a parametric uncertainty in the objective function, in this case we don't and I don't think we will

In [ ]:
# solve the mpLP

from ppopt.mpqp_program import MPLP_Program
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm

prog = MPLP_Program(A, b, c, H, CRa, CRb, F)

solution = solve_mpqp(prog, mpqp_algorithm.combinatorial)

In [ ]:
# get the critical regions

region_list = solution.critical_regions

thetas = []
constants = []
for i in range(len(region_list)):
    A = region_list[i].A
    b = region_list[i].b
    theta_coeff = c.T@A
    constant_coeff = c.T@b
    thetas.append(theta_coeff[0][0])
    constants.append(constant_coeff[0][0])

In [ ]:
region_list

In [ ]:
# numerical check of the cost vs flex curve
for t in np.linspace(list(delta_dict.items())[0][1] + 0.001, list(delta_dict.items())[-1][1]+0.001):
    t_new = np.array([[t]])
    p_sol = solution.evaluate_objective(t_new)

    plt.scatter(t, p_sol)
plt.xlabel('Flexibility index')
plt.ylabel('Cost')

In [ ]:
flex_index = [0.5, 0.75, 1]
d1 = [0, 0.375, 1.33]
d2 = [0,0,1.33]

plt.plot(flex_index, d1, label = 'd1')
plt.plot(flex_index, d2, label = 'd2')
plt.xlabel('Flexibility index')
plt.ylabel('Design variable increase needed')
plt.legend()